# Factorio 物流机器人流量计算

In [1]:
import numpy as np
from scipy import optimize as op
import sympy as sp
import pandas as pd
from IPython.display import display, Image, Math, Markdown

np.set_printoptions(precision=6, suppress=True, floatmode='maxprec')
pd.set_option('display.precision', 2)

# 数据
- 游戏中没有具体的长度单位。本文为了方便，记`1格 = 1m`
- 物流机器人基础移动速度为`3 m/s`，可由无限科技`科技：作业机器人速度`提升`240% + 65%*(Level-5)`倍，设最终的移动速度为 v
- 每次运输携带物品量`n`由`科技：作业机器人货物运量`提升。文中按最高值`n = 4`进行计算
- 飞行能耗为单位时间 $P_t = 3 kJ/s$且单位距离$P_s = 5 kJ/m$，但随着`科技：作业机器人速度`的提升，前者相对后者来说越来越微不足道
- 机器人蓄电量随品质提升，从普通级的`1.5 MJ`到传说级的`9 MJ`，电量降至`20%`时必须返回机器人指令平台充电。设此蓄电量为A
- 一台机器人指令中心有4个充电口，每个提供的充电速度从普通级的`500 kW`到传说级的`1.25 MW`，总充电速度为`2 MW`到`5 MW`，设此总充电速度为C

# 建模

## 1. 能耗
- 将单位时间能耗也折算为距离，则机器人实际单位距离能耗为$P_S = P_t / v + P_s$。`科技：作业机器人速度`只需达到5级，前者便已不足后者的10%。因此随着该科技的提升，在近似计算中可以忽略前者，认为 $P_S \approx P_s = 5 kJ/m$
- 此时机器人最大搬运距离 $S_{max} = 0.8A / P_S$, 普通级为`240 m`, 传说级为`1440 m`
- 若将单位距离能耗折算为时间，则$P_T = P_t + P_s \cdot v \approx P_s v$

## 2. 运载效率
- 假设在繁忙情况下，充满电的机器人一次完整的行程为 $C_1 \xrightarrow{空载} A_1 \xrightarrow{满载} B_1 \xrightarrow{空载} A_2 \xrightarrow{满载} B_2 \dots \rightarrow C_2$，其中$A_i$为货物来源，$B_i$为货物目的，$C_i$为充电站。记满载行程占总行程的比例为$K$, 则
$$K = \frac{\sum A_i B_i}{C_1 A_1 + \sum A_i B_i + \sum B_i A_{i+1}} = \frac{S_满}{S_充 + S_满 + S_空}$$
- 由于机器人电量消耗80%后就需前往充电，故$S_充 + S_满 + S_空 \approx  S_{max} = 0.8A / P_S$，而最后一段$X_n\rightarrow C_2$不应计入行程
- $K$也可以视为机器人对能量利用的效率。可以看出，在产线不变的情况下，充电站离供货箱越近、机器人品质提高（蓄电量提高），可以有效减小$S_充$的占比，使能量利用率提高。
- 近似估计$K$的范围：假定充电行程相对运载行程可以忽略不计，又由于平均情况下$B_i A_{i+1} \le A_i B_i$（否则机器人直接在$A_i B_i$间往返效率更高），则有$0.5 \le K \lt 1$


## 3. 单位时间总行程
- 统计产线在生产稳定时单位时间内消耗和生产出的每种物品数目为$n_i$，各自需由机器人运输的距离为$d_i$，则产线在设计上就需要在单位时间内具有总运载量(即吞吐量) $N = \sum n_i$（单位：件/秒）和运载距离$D = \sum n_i d_i$（单位：件·米/秒）
- 定义 $ L = D / N = \sum n_i d_i / \sum n_i $ 为产线的`平均运载距离`，显然该值只与产线的物流箱布置有关，当产线成比例扩大或者提速时保持不变
- 考虑到机器人运载量为$n$，则需单位时间所有机器人的运载行程$S_满 = D/n$，总行程$S = S_满 / K = D / (n K)$


## 4. 输入功率与吞吐量
- 完成该行程$S$的需要的总能量为$P = P_S \cdot S = (D P_S)/(n K)$
- 由于机器人电量消耗掉80%后就需前往充电，设设该某区域有$N_C$台机器人指令中心向机器人提供充电，每台提供的总充电功率为$C_i$，则单位时间内$P$应恰好等于80%的总充电功率$C( = \sum C_i)$
- 再代入$D = L N$有 $P = 80\% C = (L N P_S) / (n K)$, 即吞吐量 $$N = \frac{80\% C \cdot n K}{L P_S} $$

# 讨论
- 式中：$L$与产线布置相关，$P_S$与`作业机器人速度`微弱相关但当速度充分大时几乎为定值，$C$与指令平台相关，$n$取决于`科技：作业机器人货物运量`，$K$则与产线和指令平台的布置、机器人品质等因素均相关。
- 因此从以下几个方面进行讨论：

## 1. 产线布置对吞吐量的影响
- 可以看出，吞吐量与产线的`平均运载距离`$L$成反比
- 同时，若能尽可能地减小空载行程的占比，则也可通过提升$K$使吞吐量得到一定提升（比如，令上一集货箱$B_i$尽可能接近下一供货箱$A_{i+1}$，同时产线整体的集-供货箱首尾成环）

## 2. 机器人指令平台对吞吐量的影响
- 平台数量越多、品质越高，能提供的$C$越大，吞吐量将成正比地增加
- 但是，若平台距产线的供货箱太远即$S_充$相对$S_{max}$不可忽略，则$K$会显著减小，导致吞吐量下降

## 3. 机器人的品质和相关科技对吞吐量的影响
- 本文中总是假设有充分的机器人参与运输。一般来讲，若要充分利用$C$的能量输入，至少需要的机器人数为$N_{robot} = C / P_T \approx C / (P_s v)$
- 也即，`科技：作业机器人速度`等级越高，所需的机器人数量越少
- 机器人的品质可提升其蓄电量，使$S_充$相对$S_{max}$更小，从而增大机器人的能量利用率$K$，使吞吐量增大
- `作业机器人货物运量`$n$直接正比例地提升吞吐量

# 数量估算
- 假设产线中货物的`平均运载距离`$L = 10 m$；使用1台传说级机器人指令平台提供运输$C = 5 MW$；运载效率未经特别优化但通过指令平台布置和使用传说级机器人使得$S_充$可忽略不计；$n$点满为4；`作业机器人速度`和机器人数量均足够高：此时
$$U = \frac{80\% C n K}{L P_S} = \frac{0.8 \cdot 5MW \cdot 4 \cdot 0.5} {10 m \cdot 5 kJ/m} = 160 /s $$
    即此条件下的吞吐量约为`160件/s`